# SIMPACK Cartographic Track 示例思路和代码
- 实现了各分段（STR/CIR/BLO）的曲率函数 κ(s)，并在分段交界处用一段“平滑区”保证二阶导数的连续——这与 SIMPACK 的“Bloss + smoothing”原理相似，可以在整个里程范围内得到一条更光滑、与 SIMPACK 结果更贴合的 κ(s)
- 最后对曲率做数值积分，即可获得轨道在平面内的坐标；若再加上竖向坡度、超高等，也能扩展到 3D 轨道
- **数据保存**: 轨道空间几何保存为 trajectory_data.npz

## 提示
1. 下述代码依然是“示例”，并未 100% 复刻 SIMPACK 的所有插值细节；尤其在 Bloss 与 smoothing 同处一段时，SIMPACK 可能还包含其他工程上累积的经验规则
2. 然而，本示例已体现用分段多项式在段交界处做无缝拼接的关键思路，能够在多数应用中和 SIMPACK 原生结果达到极高的一致度
3. 代码示例只对“水平曲率κ”演示了完整的拼接；若要处理超高 𝑢(𝑠)，请使用相同的逻辑（把 κ 换成 u 即可）

# 整体思路
## 1. 原始分段函数
在 Cartographic Track 的每段，都可以先写出一个原始曲率公式 κ_raw(s)：
- STR 段：κ=0
- CIR 段：κ=1/R​（常数）
- BLO 段（Bloss 过渡曲线）
  $\kappa(s) = \kappa_1 + (\kappa_2 - \kappa_1) \cdot (3x^2 - 2x^3), \quad x = \frac{s}{L_\mathrm{seg}}$。

其中, $\kappa_1 = \frac{1}{R_1}, \quad \kappa_2 = \frac{1}{R_2}$。
例如，若一段声明 ('BLO',50,0,300)，则表示“Bloss 过渡曲线长度 50m，初末曲率分别 0 和 1/300”。

## 2. 分段连接 + Smoothing

SIMPACK 提供一个"平滑总长度" $L_{\mathrm{smo}}$（常见取 2 m ～ 5 m 或更小）来**数值上**保证相邻两段衔接时的**高阶连续**：

* 在分段 $i$ 和 $i+1$ 的边界 $s_i$ 附近，**各留一半 $L_{\mathrm{smo}}/2$ 给平滑过渡**。
* 这部分平滑区并**不**额外增加段长，而是**覆盖**了分段的首尾各 $L_{\mathrm{smo}}/2$ 范围。
* 用一条**5 次多项式**在 $[\,s_i - L_{\mathrm{smo}}/2,\; s_i + L_{\mathrm{smo}}/2\,]$ 上插值，使其与前后段的"原始" $\kappa_{\mathrm{raw}}$ 的 $\kappa$、$\kappa'$、$\kappa''$ 在边界上保持一致，从而保证 $\mathcal{C}^2$ 光滑。

这样，一条具名义长度 $L_{\mathrm{seg}}$ 的 Bloss 段，其真正用于「Bloss 公式」的区间只剩 $(L_{\mathrm{seg}} - L_{\mathrm{smo}})$；首尾的 $L_{\mathrm{smo}}/2$ 用于 polynomial smoothing。**结果**：

* 总长仍是 $L_{\mathrm{seg}}$，但在内部融合了 Bloss 与 smoothing 的小段。
* 段与段之间二阶导数连续，避免了"反复过渡"所带来的偏移累积。

## 3. 生成全局 $\kappa(s)$ 函数

* 将**所有**段（STR/CIR/BLO）加上其**前后 smoothing** "拼"起来后，就得到一个从 $s=0$ 到 $s=L_{\mathrm{total}}$ 的分段式函数 $\kappa_{\mathrm{global}}(s)$。
* 同理，可对超高 $u(s)$ 或竖向坡度 $p(s)$ 做类似的 piecewise + smoothing 处理。

## 4. 数值积分得到 $(x(s),\,y(s))$

一旦有了 $\kappa_{\mathrm{global}}(s)$，即可做**航向角** $\psi$ 的积分：$\frac{d\psi}{ds} \;=\;\kappa(s) \quad\Longrightarrow\quad \psi(s)\;=\;\int_0^s \kappa(\tau)\,d\tau.$

然后在平面内积分坐标：$\frac{dx}{ds} = \cos\bigl[\psi(s)\bigr], \qquad \frac{dy}{ds} = \sin\bigl[\psi(s)\bigr].$

若仅考虑水平面（无竖曲线），则可令 $z=0$。若要考虑**坡度** $p(s)$，再做一次积分 $z'(s)=p(s)$ 得到 $z(s)$。

## 5. 左右钢轨 + 3D

* 如果还要得到左右钢轨的 3D 坐标，可先在每个 $s$ 点计算**超高** $u(s)$ 并转换成横滚角 $\phi(s)=\arcsin\!\bigl[u(s)/b_{\mathrm{ref}}\bigr]$。
* 然后，以中心线为基准，在局部坐标 $(y_{\mathrm{local}},\,z_{\mathrm{local}})$ 上做 $\pm b_{\mathrm{ref}}/2$ 与 $\pm u/2$ 的偏移，通过 $\psi(s)$ 和 $\phi(s)$ 转回全局，便能得到左右股钢轨或轮轨实际位置。
* 在已封装的 Python 代码中，这往往体现在"left_rail"与"right_rail"的计算环节中。

## 小结

1. **"Bloss + smoothing"** 的关键是：smoothing 并不额外拉长段，而是**占用**分段首尾的小范围，用高阶多项式衔接前后段的曲率及其导数。
2. 在此思路下，**整条轨道**得到一个二阶（甚至三阶）连续的 $\kappa(s)$ 函数，避免了"重复过渡"或"段间不连续"导致的误差累积。
3. **示例代码**（参考 xxx.py）中，演示了如何将**原始 Bloss** 段 + **多项式平滑**合并到一起，并通过数值积分得到平面曲线；可按同理处理超高、竖曲线等需求。
4. 由于 SIMPACK 还可能在极端工况（小半径、陡坡、大超高）时有其他内部修正，若需与其"1:1 完全对齐"，还请检查文档细节或对照实际导出的轨道数据做比对。

# 独立运行代码
- 定义轨道分段及其参量
- 输出 .npz 与 .json 轨道数据

In [ ]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
import json
import os
import platform

# 从 tools_SPCKTrk.py 中导入主函数
from tools_SPCKTrk import read_track_segments
from tools_SPCKTrk import generate_trajectory_withVertical
                           
#===================== 1) 轨道段定义 =====================
"""
# 硬编码定义

# 水平段
h_segments = [
    ('STR',  50      ),            # 1.直线50m
    ('BLO',  50, 0.0, 300.0 ),     # 2.Bl.  0-->1/300
    ('CIR', 1000, 300 ),           # 3.圆曲线 300m半径
    ('BLO',  50, 300, 0 ),         # 4.Bl.  1/300-->0
    ('STR',  500     )             # 5.直线500m
]

# 超高段
u_segments = [
    ('CST', 50,   0.0),            # 常值超高 0
    ('BLO', 50, 0.0, 0.12),        # 从0变化到0.12
    ('CST', 1000, 0.12),           # 保持0.12不变
    ('BLO', 50, 0.12, 0.0),        # 回到0
    ('CST', 500, 0.0 )
]

"""

# 1) 设置平滑段总长度、轨距、步长等参数
L_smo = 3.0     # => 左右各1.5m 用于水平曲线/超高拼接
b_ref = 1.5     # 中心线到钢轨的半轨距

# 积分步长 ds 取 0.5 ~ 10 之间，默认为 0.5
"""
详细说明用途:
 - ds 取 0.1: Ubuntu 端生成待插值 SPCK 线路，可以在 UE5 建模末期离线确认线路建模精确度
 - ds 取 0.1 ~ 0.5: 用于与 SPCK 所生成轨道平面图的高精度对比，导入 UE5 在线会极其卡顿
 - ds 取 0.5: UE5 在线生成约需 30s；离线生成极不推荐，会生成造成严重卡顿的巨大几何体
 - ds 取 1: UE5 在线渲染中将出现跳帧、抽动运行的现象，依旧不推荐用于 UE5 的离线生成，与 ds=0.5 在曲线上可以发现几何体的区别
 - ds 取 5: ROS2 节点的位置求解插值精度将较低，UE5 中将发生明显的“毛毛虫式”点动跳帧运行的现象
 - ds 取 10: 可以用于 UE5 离线生成用于测试的轨道几何体，其他用途均不推荐
 - ds 大于 10: 本程序无法求解出轨道线路
"""
# UE5 生成轨道时默认取 0.5，并可与 ds = 0.1 ROS2节点配合
# Windows UE5 在线生成轨道（JSON由0.5生成）== UDP == Ubuntu ROS2（JSON由0.1生成）

if platform.system() == 'Windows':
    ds = 0.5  # Windows系统
else:
    ds = 0.1  # Linux/Ubuntu系统
print(f"当前操作系统: {platform.system()}, ds值: {ds}")

# 2) 读取轨道分段定义(Excel或其它来源)
file_path = r"../docs/跨线运行SPCK曲线定义组.xlsx"
h_segments, u_segments, v_segments = read_track_segments(file_path)

# 3) 调用带竖向坡度的生成函数
trajectory_data, Kappa, U, Slope = generate_trajectory_withVertical(
    h_segments=h_segments,
    u_segments=u_segments,
    v_segments=v_segments,
    L_smo=L_smo,
    b_ref=b_ref,
    ds=ds,
    z0=0.0
)

# 4) 从 trajectory_data 中取出数值
s_vals     = np.array(trajectory_data['s'])      # [0, Lend]
xvals      = np.array(trajectory_data['x'])
yvals      = np.array(trajectory_data['y'])
zvals      = np.array(trajectory_data['z'])
psi_vals   = np.array(trajectory_data['psi'])
phi_vals   = np.array(trajectory_data['phi'])
left_rail  = np.array(trajectory_data['left_rail'])
right_rail = np.array(trajectory_data['right_rail'])

# 5) 在初始位置 X负方向延伸一段 s in [-15, 0), 并让kappa=0, phi=0, slope=0
extension_length = 15.0
num_new_points = int(extension_length / ds) + 1
s_extension = np.linspace(-extension_length, 0, num_new_points)
s_extension = s_extension[:-1]  # 移除最后一个点，避免与原轨起点重复

# 对应 x= s_extension, y=0, z=0, psi=0, phi=0
x_extension = s_extension.copy()
y_extension = np.zeros_like(s_extension)
z_extension = np.zeros_like(s_extension)
psi_extension = np.zeros_like(s_extension)
phi_extension = np.zeros_like(s_extension)

# 计算左右钢轨(若想保留)
left_rail_extension = []
right_rail_extension = []
half_b = b_ref/2.0
for i in range(len(s_extension)):
    xC = x_extension[i]
    yC = y_extension[i]
    zC = z_extension[i]
    # psi=0 => 横向偏移仅在y方向
    dxL = 0
    dyL = half_b
    xL  = xC + dxL
    yL  = yC + dyL
    zL  = zC
    dxR = 0
    dyR = -half_b
    xR  = xC + dxR
    yR  = yC + dyR
    zR  = zC

    left_rail_extension.append((xL,yL,zL))
    right_rail_extension.append((xR,yR,zR))

left_rail_extension  = np.array(left_rail_extension)
right_rail_extension = np.array(right_rail_extension)

# 6) 合并新旧数据
s_vals   = np.concatenate([s_extension, s_vals])
xvals    = np.concatenate([x_extension, xvals])
yvals    = np.concatenate([y_extension, yvals])
zvals    = np.concatenate([z_extension, zvals])
psi_vals = np.concatenate([psi_extension, psi_vals])
phi_vals = np.concatenate([phi_extension, phi_vals])
left_rail  = np.vstack([left_rail_extension, left_rail])
right_rail = np.vstack([right_rail_extension, right_rail])

# 6.1) 重新构建 kappa_vals, u_vals, 以及 slope_vals
kappa_list = []
u_list = []
slope_list = []

for s_ in s_vals:
    if s_ < 0:
        # 在[-15,0) 段，kappa=0, u=0, slope=0
        kappa_list.append(0.0)
        u_list.append(0.0)
        slope_list.append(0.0)
    else:
        kappa_list.append( Kappa(s_) )
        u_list.append( U(s_) )
        slope_list.append( Slope(s_) )  # 由generate_trajectory_withVertical返回的Slope函数

kappa_vals = np.array(kappa_list)
u_vals     = np.array(u_list)
slope_vals = np.array(slope_list)

# 7) 更新 trajectory_data: 
#    这里在JSON中新增 "slope" 字段 
trajectory_data = {
    's':     s_vals.tolist(),
    'x':     xvals.tolist(),
    'y':     yvals.tolist(),
    'z':     zvals.tolist(),
    'psi':   psi_vals.tolist(),
    'phi':   phi_vals.tolist(),
    'left_rail':  left_rail.tolist(),
    'right_rail': right_rail.tolist(),
    # 新增
    'slope': slope_vals.tolist()
}

# 8) 存储到文件
np.savez('trajectory_data.npz', **trajectory_data)

with open('trajectory_data.json', 'w') as jf:
    json.dump(trajectory_data, jf, indent=4)

print("轨道数据已生成并保存到 'trajectory_data.json' 与 'trajectory_data.npz'.")

#===================== 4) 绘制3D轨迹视图 =====================
fig = plt.figure(figsize=(9, 18))

#----- (4.1) 3D视图(不等比例) -----
ax1 = fig.add_subplot(311, projection='3d')
ax1.plot(xvals, yvals, zvals, 'k-', label='CenterLine')
ax1.plot(left_rail[:,0], left_rail[:,1], left_rail[:,2], 'r-', label='Left Rail')
ax1.plot(right_rail[:,0], right_rail[:,1], right_rail[:,2], 'b-', label='Right Rail')
ax1.set_title("3D Track View (Non-Equal Axes)")
ax1.set_xlabel("X [m]")
ax1.set_ylabel("Y [m]")
ax1.set_zlabel("Z [m]")
ax1.legend()

#----- (4.2) 3D视图(等比例) -----
ax2 = fig.add_subplot(312, projection='3d')
ax2.plot(xvals, yvals, zvals, 'k-')
ax2.plot(left_rail[:,0], left_rail[:,1], left_rail[:,2], 'r-')
ax2.plot(right_rail[:,0], right_rail[:,1], right_rail[:,2], 'b-')
ax2.set_title("3D Track View (Equal Axes)")
ax2.set_xlabel("X [m]")
ax2.set_ylabel("Y [m]")
ax2.set_zlabel("Z [m]")

# 强制坐标轴等比例
x_limits = ax2.get_xlim3d()
y_limits = ax2.get_ylim3d()
z_limits = ax2.get_zlim3d()
x_range = abs(x_limits[1] - x_limits[0])
y_range = abs(y_limits[1] - y_limits[0])
z_range = abs(z_limits[1] - z_limits[0])
max_range = max(x_range, y_range, z_range)
mid_x = 0.5*(x_limits[0] + x_limits[1])
mid_y = 0.5*(y_limits[0] + y_limits[1])
mid_z = 0.5*(z_limits[0] + z_limits[1])
ax2.set_xlim(mid_x - max_range/2, mid_x + max_range/2)
ax2.set_ylim(mid_y - max_range/2, mid_y + max_range/2)
ax2.set_zlim(mid_z - max_range/2, mid_z + max_range/2)

#----- (4.3) XY平面俯视图(2D) -----
ax3 = fig.add_subplot(313)
ax3.plot(xvals, yvals, 'k-', label='CenterLine')
ax3.plot(left_rail[:,0], left_rail[:,1], 'r-', label='Left Rail')
ax3.plot(right_rail[:,0], right_rail[:,1], 'b-', label='Right Rail')
ax3.set_aspect('equal', 'box')
ax3.set_title("XY Plane View (Top-Down)")
ax3.set_xlabel("X [m]")
ax3.set_ylabel("Y [m]")
ax3.legend()
ax3.grid(True)

plt.tight_layout(pad=2.0)
plt.show()

# 自主作图与 SIMPACK 原始数据对比
- 需要**先运行**上述作图函数, 以获得 left_rail, right_rail, xvals, yvals 
- SIMPACK GUI 之中，轨道平面图之中，Y在横坐标，X在纵坐标
- **读取** SPCK 导出的水平直线与曲线线路分布的数据文件: TrkHorizontal_R300m60kmph_Vehicle4WDB.txt

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

from tools_SPCKTrk import read_track_data

# 读取SIMPACK轨道数据
# 短线路 superlev_file = os.path.join(os.getcwd(), 'TrkHorizontal_R300m60kmph_Vehicle4WDB.txt')
# 完整跨网运行线路 superlev_file = os.path.join(os.getcwd(), 'TrkHorizontal_VirtualLine.txt')

simpack_file = os.path.join(os.getcwd(), 'TrkHorizontal_VirtualLine.txt')
splined_track_x, splined_track_y = read_track_data(simpack_file)

# 创建独立的图形来对比数据
plt.figure(figsize=(10, 8))
plt.plot(yvals, xvals, 'b-*', linewidth=2, label='Our CenterLine')
plt.plot(splined_track_x, splined_track_y, 'r--', linewidth=2, label='SIMPACK Track')
plt.xlabel("Y [m]")
plt.ylabel("X [m]")
plt.title("Horizontal Track Comparison: Our Implementation vs SIMPACK")
plt.legend()
plt.axis('equal')  # 设置坐标轴等比例
plt.grid(True)     # 增加网格线
plt.tight_layout()
plt.show()

# SIMPACK超高轨道数据可视化
- 红色为 SPCK 导出
- 蓝色为自定义曲线
- 两者有极小误差, 小于 2e-5 m（0.02 mm）
- SPCK 导出的超高随线路延伸的数据文件: TrkSuperlev_R300m60kmph_Vehicle4WDB.txt

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 定义轨道参数
h_s_bounds = [ 20 * 1000 ]  # 轨道总长，例如 20 km
ds = ds # 0.1

# 1. 生成要绘图的 s 序列
s_end = h_s_bounds[-1]
s_vals = np.arange(-extension_length, s_end + ds*0.1, ds) # s_vals = np.arange(-10, s_end + ds*0.1, ds)

# 2. 对每个 s 计算超高 u(s)
u_vals = [U(s) for s in s_vals]

# 读取SIMPACK超高轨道数据
# 短线路 superlev_file = os.path.join(os.getcwd(), 'TrkSuperlev_R300m60kmph_Vehicle4WDB.txt')
# 完整跨网运行线路 superlev_file = os.path.join(os.getcwd(), 'TrkSuperlev_VirtualLine.txt')

superlev_file = os.path.join(os.getcwd(), 'TrkSuperlev_VirtualLine.txt')
splined_track_x, splined_track_y = read_track_data(superlev_file)

plt.figure(figsize=(10, 6))

# 用蓝色绘制自定义曲线
plt.plot(s_vals, u_vals, 'b-', linewidth=2, label='Our Superelevation u(s)')

# 用红色虚线绘制SIMPACK数据以提高可见性
plt.plot(splined_track_x, splined_track_y, 'r--', linewidth=2, label='SIMPACK Superelevation')

# 保持图例和标签为英文
plt.title("Superelevation Comparison")
plt.xlabel("s [m]")
plt.ylabel("u(s) [m]")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


# 读取 SIMPACK 导出的竖向轨道数据 与 自己生成的中心线与左右钢轨垂向数据 进行可视化对比
### read_track_Zs_data 专门用于读取 SIMPACK 导出的竖向轨道数据
### 自己生成的 (s,z) 
1. 中心线
- 里程：s_vals = np.array(trajectory_data['s'])
- 垂向高度：our_center_z = np.array(trajectory_data['z'])
2. 左轨
- 里程：与中心线相同，可继续用 s_vals 视为离散点；
- 垂向高度：our_left_z = left_rail[:, 2]
3. 右轨
- 里程：同样用 s_vals；
- 垂向高度：our_right_z = right_rail[:, 2]
### SPCK 导出时选择 z(s)，并且需要在 Plots-Layout 模块中选择 center - Enabled、sides - Enabled

In [ ]:
%matplotlib widget
import numpy as np
import json
import os

# 从 tools_SPCKTrk.py 中导入 Zs 读取函数
from tools_SPCKTrk import read_track_Zs_data

# 1) 先读取 SIMPACK 竖曲线数据
simpack_file = os.path.join(os.getcwd(), 'TrkZs_sides_VirtualLine.txt')
(center_s, center_z, right_s, right_z, left_s, left_z) = read_track_Zs_data(simpack_file) # 从SIMPACK导出的 z(s)

print(f"SIMPACK 导出的中心线数据点数: {len(center_s)}")
print(f"SIMPACK 导出的右股轨道数据点数: {len(right_s)}")
print(f"SIMPACK 导出的左股轨道数据点数: {len(left_s)}")

# 自己生成的中心线与左右钢轨垂向数据
s_vals    = np.array(trajectory_data['s'])
zvals  = np.array(trajectory_data['z'])
left_rail[:,2]    = np.array(trajectory_data['left_rail'])[:,2]
right_rail[:,2]   = np.array(trajectory_data['right_rail'])[:,2]


# 3) 绘图对比
fig = plt.figure(figsize=(8, 10))

# ========== (3.1) 中心线 z(s) 对比 ==========
ax1 = fig.add_subplot(311)
ax1.plot(s_vals, zvals, 'b-', label='Our Center Z(s)')
ax1.plot(center_s, center_z, 'r--', label='SIMPACK Center')
ax1.set_xlabel('s [m]')
ax1.set_ylabel('Z (Center) [m]')
ax1.set_title('Center Line Z(s) Comparison')
ax1.grid(True)
ax1.legend()

# ========== (3.2) 左股轨道 z(s) 对比 ==========
ax2 = fig.add_subplot(312)
ax2.plot(s_vals,  right_rail [:,2], 'b-', label='Our Left Rail Z(s)') # 此处左右颠倒，但图中结果完全一致
ax2.plot(left_s, left_z, 'r--', label='SIMPACK Left Rail')
ax2.set_xlabel('s [m]')
ax2.set_ylabel('Z (Left Rail) [m]')
ax2.set_title('Left Rail Z(s) Comparison')
ax2.grid(True)
ax2.legend()

# ========== (3.3) 右股轨道 z(s) 对比 ==========
ax3 = fig.add_subplot(313)
ax3.plot(s_vals, left_rail[:,2], 'b-', label='Our Right Rail Z(s)')
ax3.plot(right_s, right_z, 'r--', label='SIMPACK Right Rail')
ax3.set_xlabel('s [m]')
ax3.set_ylabel('Z (Right Rail) [m]')
ax3.set_title('Right Rail Z(s) Comparison')
ax3.grid(True)
ax3.legend()

plt.tight_layout(pad=2.0)
plt.show()


# 占位

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd
import json
import platform
import math

# 从 tools_SPCKTrk.py 中导入所需的工具函数
from tools_SPCKTrk import read_track_segments
from tools_SPCKTrk import generate_trajectory_withVertical
from tools_SPCKTrk import build_s_bounds

def sample_track_points_with_yaw_pitch_cm_and_segments(
    trajectory_data,
    segments,
    sample_step=25.0,
    merge_tol=1e-8,
    out_csv="TrackSamples_TrainTemplate.csv"
):
    """
    在已有轨道数据(trajectory_data)上按里程每隔 sample_step 米进行采样，
    并额外对每个分段的起止里程点进行插值。
    之后去除同一坐标/姿态的重复行(合并annotation)，并计算相邻点的rel_yaw, rel_pitch。

    参数
    ----------
    trajectory_data : dict
        包含 's','x','y','z' 四个字段的轨迹数据(单位:米)，以及可选 'psi','phi' 等信息。
        如: trajectory_data['s'] -> [0,1,2,...], trajectory_data['x'] -> [...], ...
    segments : list
        类似 [ ('STR', 300), ('BLO', 27.5, 0, -1500), ... ] 的平面段信息。
    sample_step : float
        每隔多少米进行均匀采样(默认25m)。
    merge_tol : float
        判断两行数据是否“重复”的容差(默认1e-8)。
    out_csv : str
        输出 CSV 文件名。

    返回
    -------
    df_final : pd.DataFrame
        含以下关键列:
          ['s_cm','x_cm','y_cm','z_cm',
           'yaw_deg','pitch_deg',
           'rel_yaw_deg','rel_pitch_deg',
           'annotation']
        并写入 out_csv.
    """

    # ====== 1) 从 trajectory_data 拿到原始离散的 s,x,y,z ======
    s_vals = np.array(trajectory_data['s'], dtype=float)
    x_vals = np.array(trajectory_data['x'], dtype=float)
    y_vals = np.array(trajectory_data['y'], dtype=float)
    z_vals = np.array(trajectory_data['z'], dtype=float)

    n = len(s_vals)
    if not (len(x_vals) == n and len(y_vals) == n and len(z_vals) == n):
        raise ValueError("trajectory_data 中 s,x,y,z 长度不一致！")

    # 若 s 不单调递增，先排序
    if not np.all(s_vals[1:] >= s_vals[:-1]):
        sort_idx = np.argsort(s_vals)
        s_vals = s_vals[sort_idx]
        x_vals = x_vals[sort_idx]
        y_vals = y_vals[sort_idx]
        z_vals = z_vals[sort_idx]

    s_min, s_max = s_vals[0], s_vals[-1]

    # ====== 2) 定义插值函数(线性) ======
    def interp_x(sq): return np.interp(sq, s_vals, x_vals)
    def interp_y(sq): return np.interp(sq, s_vals, y_vals)
    def interp_z(sq): return np.interp(sq, s_vals, z_vals)

    # ====== 3) 等步长采样列表 ======
    sample_s_list = []
    curr_s = s_min
    while curr_s <= s_max + 1e-9:
        sample_s_list.append(curr_s)
        curr_s += sample_step
    sample_s_arr = np.array(sample_s_list, dtype=float)

    # ====== 4) 分段边界采样 ======
    s_bounds = build_s_bounds(segments)  # [0, seg1End, seg2End, ...]
    boundary_rows = []
    for i_seg in range(len(segments)):
        seg_idx = i_seg + 1
        sbegin = s_bounds[i_seg]
        send   = s_bounds[i_seg + 1]

        if s_min <= sbegin <= s_max:
            xB = interp_x(sbegin)
            yB = interp_y(sbegin)
            zB = interp_z(sbegin)
            boundary_rows.append({
                's': sbegin,
                'x': xB,
                'y': yB,
                'z': zB,
                'annotation': f"分段{seg_idx} 起始点"
            })
        if s_min <= send <= s_max:
            xE = interp_x(send)
            yE = interp_y(send)
            zE = interp_z(send)
            boundary_rows.append({
                's': send,
                'x': xE,
                'y': yE,
                'z': zE,
                'annotation': f"分段{seg_idx} 终止点"
            })

    df_boundary = pd.DataFrame(boundary_rows, columns=['s','x','y','z','annotation'])

    # ====== 5) 计算等步长采样点处 (x,y,z,yaw,pitch) ======
    #     yaw/pitch 用中心差分(±0.5m)估计
    def clamp(value, lo, hi):
        return max(lo, min(value, hi))

    half_delta = 0.5  # yaw/pitch 用±0.5m做中心差分

    sample_rows = []
    for s_i in sample_s_arr:
        xC = interp_x(s_i)
        yC = interp_y(s_i)
        zC = interp_z(s_i)

        # 中心差分计算 yaw, pitch
        s_left  = clamp(s_i - half_delta, s_min, s_max)
        s_right = clamp(s_i + half_delta, s_min, s_max)
        xL, yL, zL = interp_x(s_left),  interp_y(s_left),  interp_z(s_left)
        xR, yR, zR = interp_x(s_right), interp_y(s_right), interp_z(s_right)
        dx = xR - xL
        dy = yR - yL
        dz = zR - zL
        if abs(dx)<1e-12 and abs(dy)<1e-12 and abs(dz)<1e-12:
            yaw   = 0.0
            pitch = 0.0
        else:
            yaw = np.arctan2(dy, dx)
            horiz_len = np.sqrt(dx*dx + dy*dy)
            pitch = np.arctan2(dz, horiz_len)

        sample_rows.append({
            's': s_i,
            'x': xC,
            'y': yC,
            'z': zC,
            'yaw': yaw,
            'pitch': pitch,
            'annotation': ""
        })

    df_samples = pd.DataFrame(sample_rows, columns=['s','x','y','z','yaw','pitch','annotation'])

    # ====== 6) 合并“分段边界”与“常规采样点” ======
    df_merged = pd.concat([df_samples, df_boundary], ignore_index=True)
    df_merged.sort_values(by='s', inplace=True)

    # 对分段边界行的 yaw,pitch 做空值填充(0.0)
    df_merged[['yaw','pitch']] = df_merged[['yaw','pitch']].fillna(0.0)

    # ====== 7) 转为 cm, 重命名 yaw->yaw_deg, pitch->pitch_deg ======
    df_merged['s_cm'] = df_merged['s'] * 100.0
    df_merged['x_cm'] = df_merged['x'] * 100.0
    df_merged['y_cm'] = df_merged['y'] * 100.0
    df_merged['z_cm'] = df_merged['z'] * 100.0
    df_merged['yaw_deg']   = df_merged['yaw'] * 180 / math.pi
    df_merged['pitch_deg'] = df_merged['pitch'] * 180 / math.pi

    # ====== 8) 去除“重复行” (merge 重复), annotation 拼接 ======
    # 判定重复行的列
    cols_check = ['s_cm','x_cm','y_cm','z_cm','yaw_deg','pitch_deg']
    def rows_close(rA, rB, tol=merge_tol):
        for c in cols_check:
            if abs(rA[c] - rB[c]) > tol:
                return False
        return True

    df_merged.reset_index(drop=True, inplace=True)
    rows_all = df_merged.to_dict('records')

    merged_rows = []
    i = 0
    while i < len(rows_all):
        rowA = rows_all[i]
        if i < len(rows_all)-1:
            rowB = rows_all[i+1]
            if rows_close(rowA, rowB, merge_tol):
                # 合并 annotation
                annA = rowA.get('annotation', "")
                annB = rowB.get('annotation', "")
                if annA and annB:
                    merged_anno = annA + " and " + annB
                else:
                    merged_anno = annA + annB  # 可能某个是空
                rowA['annotation'] = merged_anno
                merged_rows.append(rowA)
                i += 2  # 跳过下一行
                continue
        merged_rows.append(rowA)
        i += 1

    df_merged = pd.DataFrame(merged_rows)

    # ====== 9) 计算相邻行的 rel_yaw_deg, rel_pitch_deg ======
    df_merged['rel_yaw_deg']   = df_merged['yaw_deg'].diff().fillna(0.0)
    df_merged['rel_pitch_deg'] = df_merged['pitch_deg'].diff().fillna(0.0)

    # ====== 10) 排列列顺序 & 输出 CSV ======
    out_cols = [
        's_cm','x_cm','y_cm','z_cm',
        'yaw_deg','pitch_deg',
        'rel_yaw_deg','rel_pitch_deg',
        'annotation'
    ]
    df_final = df_merged[out_cols]

    df_final.to_csv(out_csv, index=False, encoding='utf-8-sig')
    print(f"采样结果已保存到: {out_csv}")

    return df_final


# =========== C) 采样输出 CSV (去重+合并+相邻姿态差分) ============
df_result = sample_track_points_with_yaw_pitch_cm_and_segments(
    trajectory_data=trajectory_data,
    segments=h_segments,
    sample_step=25.0,
    merge_tol=1e-8,
    out_csv="TrackSamples_TrainTemplate.csv"
)

    # print(df_result.head(20))
